<a href="https://colab.research.google.com/github/swastiknath/Coursera_Capstone/blob/master/Scrapping%20From%20the%20Web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>A STUDY OF THE NEIGHBOURHOODS IN CITY OF TORRONTO</h1>

<p> In this notebook, I acquire neighbourhoods data of the city of Torronto by scrapping a Wikipedia page, <a href = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M">List of Postal Codes of Canada</a> and have cleaned the unassigned values and created a dataset similar to the Assignment Guidelines.</p>


<h2> Importing the Required Liabraries </h2>

In [1]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json

from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium
from bs4 import BeautifulSoup
print('Import Success......')
import csv

Import Success......


<h2> Declaring the URL of the page and requesting HTML response </h2>
<p> Here I, request for the html response from the website and parse the response with HTML5LIB package. </p>

<p> It's worth to note here that, re-executing the following code several times can raise an exception in the <b>'Fetching the Components'</b> cell if you executing it in an online environment.</p>

```
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)
```
<p> The above error can be avoided by repeatedly executing the cell several times or by restarting the Kernel </p>

In [0]:
# Scraping the Wikipedia Webpage:
web_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
web_request = requests.get(web_url)

soup = BeautifulSoup(web_request.content, 'html5lib')
neighbourhoods=[]

<h2>Fetching the Components</h2>
<p> In the cell below, I fetch the necessary neighbourhood data from the HTML code and print the list containning all the neighbourhood dictionaries. In the printout I see many unassigned values, they need to be dropped.</p>

In [87]:
table = soup.find('table', attrs = {'class': 'wikitable sortable'})
soup.prettify()
for row in table.findAll('tr')[1:]: 
  col = row.findAll('td')
  neighbour={}
  neighbour['Neighbourhood'] = str(col[2].get_text())
  neighbour['Borough'] = str(col[1].get_text())
  neighbour['Pincode'] = str(col[0].get_text())
  neighbourhoods.append(neighbour)
  print(neighbourhoods)    

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



<h2> Dropping the unassigned values. </h2>
<p> Here I drop the unassigned values from the dataset and create a clean dataset. But here I see an encoding error. In the <b>Neighbourhood</b> column there are extra <b> \n </b> added, which needs to be eliminated. </p>

In [88]:
neighbourhoods = pd.DataFrame(neighbourhoods)
neighbourhoods.replace('Not assigned', np.nan, inplace = True)
neighbourhoods.replace('Not assigned\n', np.nan, inplace = True)
fd=neighbourhoods.dropna(axis = 0)
fd

,Borough,Neighbourhood,Pincode
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A
5,Downtown Toronto,Regent Park\n,M5A
6,North York,Lawrence Heights\n,M6A
7,North York,Lawrence Manor\n,M6A
10,Etobicoke,Islington Avenue\n,M9A
11,Scarborough,Rouge\n,M1B
12,Scarborough,Malvern\n,M1B
14,North York,Don Mills North\n,M3B


<h2> Eliminating the encoding error</h2>
<p> Here I use a python for iteration to replace the certain charcters to obtain the clean dataset. </p>

In [69]:
b = '\n'
values = []
for value in fd['Neighbourhood'].values:
  for f in b:
    value = value.replace(f, "")
    values.append(value)
print(values)
fd['Neighbourhood'] = values
fd

['Parkwoods', 'Victoria Village', 'Harbourfront', 'Regent Park', 'Lawrence Heights', 'Lawrence Manor', 'Islington Avenue', 'Rouge', 'Malvern', 'Don Mills North', 'Woodbine Gardens', 'Parkview Hill', 'Ryerson', 'Garden District', 'Glencairn', 'Cloverdale', 'Islington', 'Martin Grove', 'Princess Gardens', 'West Deane Park', 'Highland Creek', 'Rouge Hill', 'Port Union', 'Flemingdon Park', 'Don Mills South', 'Woodbine Heights', 'St. James Town', 'Humewood-Cedarvale', 'Bloordale Gardens', 'Eringate', 'Markland Wood', 'Old Burnhamthorpe', 'Guildwood', 'Morningside', 'West Hill', 'The Beaches', 'Berczy Park', 'Caledonia-Fairbanks', 'Woburn', 'Leaside', 'Central Bay Street', 'Christie', 'Cedarbrae', 'Hillcrest Village', 'Bathurst Manor', 'Downsview North', 'Wilson Heights', 'Thorncliffe Park', 'Adelaide', 'King', 'Richmond', 'Dovercourt Village', 'Dufferin', 'Scarborough Village', 'Fairview', 'Henry Farm', 'Oriole', 'Northwood Park', 'York University', 'East Toronto', 'Harbourfront East', 'Tor

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Borough,Neighbourhood,Pincode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
10,Etobicoke,Islington Avenue,M9A
11,Scarborough,Rouge,M1B
12,Scarborough,Malvern,M1B
14,North York,Don Mills North,M3B


<h2> Observing Uniqueness in the datset </h2>

In [70]:
final_df = fd[['Pincode', 'Borough', 'Neighbourhood']]
print('Observation 1: There are {} unique Boroughs'.format(len(final_df['Borough'].unique())))

Observation 1: There are 10 unique Boroughs


<h2> Obtainning the Final Dataframe. </h2>
<p> Here I use the groupby method to sort the table with Pincodes and join the Neighbourhoods falling into the same pincode. </p>

In [74]:
fdg = fd.groupby('Pincode')['Neighbourhood', 'Borough'].agg(lambda x: ', '.join(set(x))).reset_index()
fdg

,Pincode,Neighbourhood,Borough
0,M1B,"Malvern, Rouge",Scarborough
1,M1C,"Rouge Hill, Port Union, Highland Creek",Scarborough
2,M1E,"Morningside, Guildwood, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough
5,M1J,Scarborough Village,Scarborough
6,M1K,"Ionview, East Birchmount Park, Kennedy Park",Scarborough
7,M1L,"Oakridge, Golden Mile, Clairlea",Scarborough
8,M1M,"Cliffcrest, Scarborough Village West, Cliffside",Scarborough
9,M1N,"Cliffside West, Birch Cliff",Scarborough


<h2> Obtainning the Shape of the Dataframe</h2>

In [81]:
the_final_dataframe = pd.DataFrame(fdg)
print("The Shape of the Final Dataframe is : ", the_final_dataframe.shape)

The Shape of the Final Dataframe is :  (102, 3)


<h2>Obtainning the number of Neighbourhoods by unique borroughs. </h2>

In [72]:
final_df.groupby('Borough').count()

,Pincode,Neighbourhood
Borough,,
Central Toronto,51,51
Downtown Toronto,111,111
East Toronto,21,21
East York,18,18
Etobicoke,135,135
Mississauga,3,3
North York,114,114
Scarborough,111,111
West Toronto,39,39


<h2> One-Hot Encoding of the Neighbourhoods in terms of the Borroughs</h2>

In [53]:
gh = final_df

one_hot = pd.get_dummies(gh[['Borough']], prefix = "", prefix_sep ="")
one_hot['PINCODE'] = gh[['Pincode']]
fixed_col = [one_hot.columns[-1]] + list(one_hot[:-1])
one_hot = one_hot[fixed_col]
one_hot

,PINCODE,Central Toronto,Downtown Toronto,East Toronto,East York,Etobicoke,Mississauga,North York,Scarborough,West Toronto,York,PINCODE
2,M3A,0,0,0,0,0,0,1,0,0,0,M3A
3,M4A,0,0,0,0,0,0,1,0,0,0,M4A
4,M5A,0,1,0,0,0,0,0,0,0,0,M5A
5,M5A,0,1,0,0,0,0,0,0,0,0,M5A
6,M6A,0,0,0,0,0,0,1,0,0,0,M6A
7,M6A,0,0,0,0,0,0,1,0,0,0,M6A
10,M9A,0,0,0,0,1,0,0,0,0,0,M9A
11,M1B,0,0,0,0,0,0,0,1,0,0,M1B
12,M1B,0,0,0,0,0,0,0,1,0,0,M1B
14,M3B,0,0,0,0,0,0,1,0,0,0,M3B


<h2> Shape of the Encoded DataFrame</h2>

In [47]:
one_hot.shape

(420, 12)